# **Data Preprocessing**

In [1]:
import pandas as pd
import nltk
import gensim
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from gensim.parsing.preprocessing import STOPWORDS
import numpy as np
import pickle
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abdul\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\abdul\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
#mounting the google drive
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
#importing the CSV file of the dataset from google drive into a dataframe
data=pd.read_csv('eclipse_platform.csv')
data

,Issue_id,Priority,Component,Duplicated_issue,Title,Description,Status,Resolution,Version,Created_time,Resolved_time
0,1,P3,Team,NaN,Usability issue with external editors (1GE6IRL),- Setup a project that contains a *.gif resour...,CLOSED,FIXED,2,2001-10-10 21:34,2012-02-09 15:57
1,2,P5,Team,NaN,Opening repository resources doesnt honor type...,Opening repository resource always open the de...,RESOLVED,FIXED,2,2001-10-10 21:34,2002-05-07 10:33
2,3,P5,Team,NaN,Sync does not indicate deletion (1GIEN83),KM (10/2/2001 5:55:18 PM); \tThis PR about the...,RESOLVED,FIXED,2,2001-10-10 21:34,2010-05-07 10:28
3,4,P5,Team,NaN,need better error message if catching up over ...,- become synchronized with some project in a r...,RESOLVED,FIXED,2,2001-10-10 21:34,2002-03-01 16:27
4,5,P3,Team,NaN,ISharingManager sharing API inconsistent (1GAU...,For getting/setting the managed state of a res...,RESOLVED,WONTFIX,2,2001-10-10 21:34,2008-08-15 8:04
...,...,...,...,...,...,...,...,...,...,...,...
85151,424679,P3,IDE,390071.0,Crash on bootstarp,NaN,CLOSED,DUPLICATE,4.3,2013-12-26 20:12,2013-12-31 5:32
85152,424714,P3,UI,NaN,[GTK/Linux] Blank Windows with GTK3,I started testing 4.4 and noticed the editor w...,CLOSED,NDUPLICATE,4.4,2013-12-28 8:57,2014-01-04 13:54
85153,424722,P3,Website,NaN,software,NaN,RESOLVED,INVALID,4.3.1,2013-12-29 3:57,2014-01-07 8:47
85154,424764,P3,SWT,NaN,Crash (MacOS) - getIvar,Process: eclipse [42412]; Path: ...,CLOSED,NDUPLICATE,4.4,2013-12-30 16:43,2013-12-31 4:35


In [4]:
#Removing the unuseful coloumns from the dataset
data = data.drop(columns=['Priority', 'Component', 'Status', 'Resolution', 'Version', 'Created_time', 'Resolved_time'], axis=1)
data

,Issue_id,Duplicated_issue,Title,Description
0,1,NaN,Usability issue with external editors (1GE6IRL),- Setup a project that contains a *.gif resour...
1,2,NaN,Opening repository resources doesnt honor type...,Opening repository resource always open the de...
2,3,NaN,Sync does not indicate deletion (1GIEN83),KM (10/2/2001 5:55:18 PM); \tThis PR about the...
3,4,NaN,need better error message if catching up over ...,- become synchronized with some project in a r...
4,5,NaN,ISharingManager sharing API inconsistent (1GAU...,For getting/setting the managed state of a res...
...,...,...,...,...
85151,424679,390071.0,Crash on bootstarp,NaN
85152,424714,NaN,[GTK/Linux] Blank Windows with GTK3,I started testing 4.4 and noticed the editor w...
85153,424722,NaN,software,NaN
85154,424764,NaN,Crash (MacOS) - getIvar,Process: eclipse [42412]; Path: ...


In [5]:
#Printing the Coloumn names
data.keys()

Index(['Issue_id', 'Duplicated_issue', 'Title', 'Description'], dtype='object')

In [6]:
#Checking the shape of the data
data.shape

(85156, 4)


## **1. Text Cleaning**

  *   Removing Invalid Reports
  *   Removing Punctuations






In [7]:
#Number of empty Description reports
data['Description'].isnull().sum()

129

In [8]:
#Number of empty Title reports
data['Title'].isnull().sum()

0

In [9]:
#removing the rows with empty Description
data = data.dropna(axis=0, subset=['Description'])

In [10]:
pd.options.display.max_colwidth = 500

In [11]:
#Showing the fixed pattern used in many reports which doesn't have to do anything with the bug itself
# Eg: fixed in head, has been marked as read only
data.loc[[4]]

,Issue_id,Duplicated_issue,Title,Description
4,5,NaN,ISharingManager sharing API inconsistent (1GAUL8H),For getting/setting the managed state of a resource; the methods are:; ; isManaged(resource) and; manage(resource); ; for getting/setting the ignore state; the methods are:; ; getIgnored(resource); setIgnored(resource); ; These should be made more consistent. I suggest renaming ignore methods:; ; isIgnored(resource) and; ignore(resource).; ; I think its good practice to not use get and set in method names unless its really; just a field accessor.; ; NOTES:


In [12]:
#Removing the fixed patterns by replacing them with " "
data["Description"]= data["Description"].str.replace("fixed in HEAD", "", case = False)
data["Description"]= data["Description"].str.replace("has been marked as readonly", " ", case = False)

<ipython-input-12-67fdc74805e6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Description"]= data["Description"].str.replace("fixed in HEAD", "", case = False)
<ipython-input-12-67fdc74805e6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Description"]= data["Description"].str.replace("has been marked as readonly", " ", case = False)


In [13]:
#Checking if it is been removed
data.loc[[4]]

,Issue_id,Duplicated_issue,Title,Description
4,5,NaN,ISharingManager sharing API inconsistent (1GAUL8H),For getting/setting the managed state of a resource; the methods are:; ; isManaged(resource) and; manage(resource); ; for getting/setting the ignore state; the methods are:; ; getIgnored(resource); setIgnored(resource); ; These should be made more consistent. I suggest renaming ignore methods:; ; isIgnored(resource) and; ignore(resource).; ; I think its good practice to not use get and set in method names unless its really; just a field accessor.; ; NOTES:


In [14]:
pd.options.display.max_colwidth = 50

In [15]:
#Text Cleaning round 1 (removing punctutions)
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('\w*\f\w*', '', text)
    text = re.sub('\(.*?\)', '', text)
    text = re.sub('\[.*]\)', '', text)
    text = text.lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [16]:
data['Description'] = data['Description'].apply(clean_text_round1)


<ipython-input-16-18cb8d411fc8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Description'] = data['Description'].apply(clean_text_round1)


In [17]:
# Apply a second round of cleaning (removing punctuations)
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\t', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [18]:
data['Title'] = data['Title'].apply(clean_text_round2)
data['Description'] = data['Description'].apply(clean_text_round2)

<ipython-input-18-6add28d453c9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Title'] = data['Title'].apply(clean_text_round2)
<ipython-input-18-6add28d453c9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Description'] = data['Description'].apply(clean_text_round2)


## **2. Stop Word Removal, Tokenization and Lemmatization**

In [19]:

def lemmatize(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 5:
            result.append(lemmatize(token))
    return result

In [20]:
data['Title'] = data['Title'].map(preprocess)
data['Description'] = data['Description'].map(preprocess)

<ipython-input-20-5cf414487d71>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Title'] = data['Title'].map(preprocess)
<ipython-input-20-5cf414487d71>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Description'] = data['Description'].map(preprocess)


In [21]:
data.to_csv("CSV/preprocessed_data.csv")

## **Seperation of Master Reports and Duplicate Reports**

In [22]:
#Number of master_reports
data['Duplicated_issue'].isnull().sum()

70629

In [23]:
#Saving all the duplicate reports into a csv file as a testing set
duplicate_reports = data.dropna(axis=0, subset=['Duplicated_issue'])
duplicate_reports.reset_index(drop=True)
duplicate_reports.to_csv('CSV/duplicate_reports.csv')

In [24]:
#Seperating all the master reports into a dataframe
master_reports = data[data.isnull().any(axis=1)]
master_reports.reset_index(drop=True)

,Issue_id,Duplicated_issue,Title,Description
0,1,NaN,"[usability, external, editors]","[project, contain, resource, release, project,..."
1,2,NaN,"[open, repository, resources, doesnt]","[open, repository, resource, default, editor, ..."
2,3,NaN,"[indicate, deletion]","[deletion, indicator, viewer, subtle, vision, ..."
3,4,NaN,"[better, message, catch, resource]","[synchronize, project, repository, different, ..."
4,5,NaN,"[isharingmanager, share, inconsistent]","[gettingsetting, manage, resource, methods, is..."
...,...,...,...,...
70624,424656,NaN,"[fail, relate, config]","[feel, relate, need, change, relate, copy, bel..."
70625,424658,NaN,"[eclipse, kepler, ubuntu, jrockit]","[eclipse, kepler, ubuntu, oracle, jrockit, att..."
70626,424672,NaN,[display],"[button, dropdown, doesnt, button, result, but..."
70627,424714,NaN,[windows],"[start, test, notice, editor, windows, package..."


In [25]:
#Removing empty master_reports after data cleaning process
# mr = pd.DataFrame()
# count = 0
# for i in range(len(master_reports)):
#     if(len(master_reports.Description[i])>2):
#         mr= mr.append(master_reports.loc[[i]])
#         count=+1

In [26]:
#Saving all the master reports into a seperate csv file
master_reports.to_csv('CSV/master_reports.csv')